In [ ]:
#pip install reportlab
import json
import glob
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import itertools
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd
import random

In [ ]:
with open('dataset.json', 'r') as file:
    data = json.load(file)

print(data)

In [ ]:
len(data)

In [ ]:
# look at json
data[0]['cuisines'][0]['label']

In [ ]:
# extract images of one restaurant
one_restaurant_info=data[0]['more_details']['full_images']
one_restaurant_images=[i['image_id'] for i in one_restaurant_info]
one_restaurant_images

In [ ]:
# do this for all restaurants
import random
res_image_dic={}
cuisine_dic={}
for res in data:
    # use cuisine type as restaurant 
    res_id= res['identifier']
    one_restaurant_info=res['more_details']['full_images']
    one_restaurant_images=[i['image_id'] for i in one_restaurant_info]
    if len(one_restaurant_images)!=0:

    # select randomly 2 images per restaurant, otherwise too many images
        selected= one_restaurant_images[-1]#random.choices(one_restaurant_images,k=1)
    res_image_dic[res_id]=selected
    cuisine_dic[res_id] = res['cuisines'][0]['label']#

In [ ]:
df_cuisine=pd.DataFrame.from_dict(cuisine_dic, orient='index').reset_index()
df_cuisine.columns=['res_id', 'cuisine']
df_cuisine.groupby('cuisine').count().sort_values(by='res_id',ascending=False).iloc[:22]
# get the country names from top 20

In [ ]:
top_22 = list(df_cuisine.groupby('cuisine').count().sort_values(by='res_id',ascending=False).iloc[:22].reset_index()['cuisine'])
top_22 # however, this will give us the top pictures in terms of both food and interior!
# change to top 22 because I want to include amazing indian food :)


In [ ]:
country=[ # 8 cuisines
 'Japanese',
 'Italian',
 'French',
 'Mediterranean Cuisine',
 'Modern British',
# 'Modern French',
# 'Cantonese',
 'Thai',
 'Chinese', 
# 'Classic French', 
    'Indian']

extract_relevant_id = list(df_cuisine[df_cuisine['cuisine'].isin(country)]['res_id'])

In [ ]:
df_cuisine_selected=df_cuisine[df_cuisine['res_id'].isin(extract_relevant_id)]

In [ ]:
df_cuisine_selected_20=pd.DataFrame()
for i in country:
    #get 50 images instead of 20
    selected=df_cuisine_selected[df_cuisine_selected['cuisine']==i].iloc[:50,:]
    df_cuisine_selected_20=pd.concat([df_cuisine_selected_20,selected],axis=0)
    
extract_relevant_id_filter=list(df_cuisine_selected_20.res_id)

In [ ]:
res_image_dic=dict((k, res_image_dic[k]) for k in extract_relevant_id_filter if k in res_image_dic)

In [ ]:
# list of all restaurant images
nested_list=list(res_image_dic.values())
image_names = nested_list.copy()
#image_names

In [ ]:
len(image_names) # okey number of images

In [ ]:
# test reading an image
folder_path = 'images'
image_path=folder_path + '/' + image_names[0] + '.jpg'
img=Image.open(image_path)
img

In [ ]:
# reading all images in a restaurant setting  (for relevant cuisines)
all_images={}
folder_path = 'images'
target_size = (256, 256) # need to resize because some images do not have the same shap

for name in image_names:
    file_name=name + '.jpg'
    image_path = folder_path + '/' + file_name
    try:
        with Image.open(image_path) as img:
            img = img.resize(target_size)
            image_array = np.array(img)

            all_images[name]=image_array
            #print(name)
    except:
        print(f"Error opening file '{file_name}', skipping...")
        continue

In [ ]:
test_image_list=[]

test_image_names=[]
for i in image_names: 
    #plt.imshow(all_images[i], cmap='gray')
    #plt.show()
    if all_images[i].shape == (256, 256, 3):
        test_image_list.append(all_images[i])
        test_image_names.append(i)
    #print(all_images[i].shape) 

In [ ]:
test=np.array(test_image_list)
test.shape

# Filter only food images using our model

In [ ]:
import keras

In [ ]:
food_interior_classifier = keras.models.load_model("food_interior_classifier.h5")
food_interior_classifier

In [ ]:
# Predict new images with the model
model_predictions=food_interior_classifier.predict(test)
# set the threshold
threshold = 0.1 # adjust this threshold to make sure what we predict as food is really food
# convert predicted probabilities to labels 0 and 1
predicted_labels = (model_predictions > threshold).astype(int)
# print the predicted labels

In [ ]:
# need to find a better threshold, many misclassifications:
# https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/

In [ ]:
# adddress class imbalance, we might want to query more images from less prevalent cuisines

In [ ]:
label_dict={0: 'food', 1: 'exterior'}
predicted=[]

food_only=[]
food_only_id=[]
for ind, item in enumerate(test): # show images in order to check prediction 
    label=predicted_labels[ind][0]
    predicted.append(label)
    if label==0:
        #print(label)
        food_only.append(item)
        food_only_id.append(test_image_names[ind])
    #print('image ' + str(ind) + ' label:' + (label_dict[label]))
    #plt.imshow(item, cmap='gray')
    #plt.show()

In [ ]:
plt.imshow(food_only[-1], cmap='gray')
plt.show()

In [ ]:
len(predicted)

In [ ]:
len(food_only)

In [ ]:
round(1- sum(predicted)/len(predicted),2)
# percent predicted as food images

In [ ]:
def save_image(filename):
    p = PdfPages(filename)
    for ind, item in enumerate(food_only): # show images in order to check prediction 
        fig= plt.figure()
        plt.imshow(item, cmap='gray')
        fig.savefig(p, format='pdf') 
        plt.title(ind)
    # close the object
    p.close()  

In [ ]:
filename = "cuisine_food_predicted.pdf"  
save_image(filename) 

In [ ]:
exterior_manual=[6, 19,23,36,37,39,47,52,54,57,61,69,72,73,74,75,79,82,85,89,96,97,98,100,
                 102,124,127,128,130,137,144,148,162,164,165,166,170,179,186,187,189,
                 190,208,212,219,222,229,258,272,280,285,289]

In [ ]:
#mannually check wrong classification (classified food as exterior)

exterior_manual=[6, 19,23,36,37,39,47,52,54,57,61,69,72,73,74,75,79,82,85,89,96,97,98,100,
                 102,124,127,128,130,137,144,148,162,164,165,166,170,179,186,187,189,
                 190,208,212,219,222,229,258,272,280,285,289]

real_food_index=[i for i in list(range(len(food_only))) if i not in exterior_manual]
len(real_food_index)

In [ ]:
real_food_image=[food_only[i] for i in real_food_index]
real_food_id=[food_only_id[i] for i in real_food_index]
for ind, item in enumerate(real_food_image): # show images in order to check prediction 
    plt.imshow(item, cmap='gray')
    plt.show()

In [ ]:
# save real food id into a txt
#with open(r'real_food_id.txt', 'w') as fp:
#    for item in real_food_id:
        # write each item on a new line
#        fp.write("%s\n" % item)
#    print('Done')

In [ ]:
final_res=dict((k, res_image_dic[k]) for k in extract_relevant_id_filter if res_image_dic[k] in real_food_id)
final_res_list=list(final_res.keys())

df_cuisine_selected[df_cuisine_selected['res_id'].isin(final_res_list)].groupby('cuisine').count()

In [ ]:
final_labels=list(df_cuisine_selected[df_cuisine_selected['res_id'].isin(final_res_list)]['cuisine'])
len(final_labels)

In [ ]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
final_labels_int= label_encoder.fit_transform(final_labels)

In [ ]:
len(real_food_image)

In [ ]:
class_cat=pd.DataFrame(final_labels)
class_cat.columns=['label']
dummies = pd.get_dummies(class_cat.label)
dummies.shape

# Modelling

In [ ]:
X = np.array(real_food_image)
y = np.array(dummies)

y = np.array(final_labels_int)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, final_labels_int, test_size=0.4, stratify=y, random_state=42)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications import VGG16

from tensorflow.keras.layers import Dropout


# Load the pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze pre-trained layers (compare with no freezing)
for layer in base_model.layers:
    layer.trainable = False

# Define your own classification layers
model = Sequential()
model.add(base_model)
model.add(Dropout(0.2)) # avoid overfitting
model.add(Flatten())


#model.add(Dense(128, activation='relu'))

#val acc>0.2
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.2)) # adding it twuice to avoid overfitting
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.2)) # adding it twuice to avoid overfitting

model.add(Dense(8, activation='softmax'))
#https://datascience.stackexchange.com/questions/39264/how-does-sigmoid-activation-work-in-multi-class-classification-problems

# Compile the model
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy','top_k_categorical_accuracy',top3_acc])

#sparse_categorical_crossentropy

In [ ]:
model.summary()

In [ ]:
import functools
#https://stackoverflow.com/questions/51951358/keras-how-to-get-top-k-accuracy
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

In [ ]:
# Train the model
#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))


# Evaluate the model on test set
score = model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

In [ ]:
# implement ways to prevent overfitting

In [ ]:
# seems no of images still not enough, or some countries have too similar dishes